## Source Code for Trained Transformer Evaluation (Main Result)

In [ ]:
from sas7bdat import SAS7BDAT
from collections import OrderedDict
import re
import os
from models import *
from eval import *
from samplers import *
from tasks import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from tqdm.notebook import tqdm
from eval import get_run_metrics, read_run_dir, get_model_from_run
# from plot_utils import *
import pickle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

%matplotlib inline
%load_ext autoreload
%autoreload 2

def evaluate_models(run_path, n_list, batch_size=64, n_dims=15, scale_Theta=1, 
                       normalize_w=True, normalize_Theta=False, mode='linear', q_truncated=None, num_task=500, 
                       delta=5, seed=1):
    # Evaluate MSE vs number of in-context samples

    torch.random.manual_seed(seed)
    p = int(1/3 * n_dims)

    # Load the GPT model
    gptmodel, conf = get_model_from_run(run_path, step=-1)
    gptmodel = gptmodel.cuda().eval()

    # Models to evaluate
    if mode == "multicollinearity-heavy":
        all_model = [RidgeIVRegressionModel(), RidgeLeastSquaresModel(), gptmodel]
    else:
        all_model = [IVRegressionModel(), LeastSquaresModel(), gptmodel]

    metrics = {}

    for model in all_model:
        metrics[model.name] = {}
        if "gpt" in model.name:
            batch_size = int(batch_size / 4)
        for n in tqdm(n_list):
            n_points = n + 1 # Include test sample
            all_pred_mse = []
            all_coef_mse = []
            metrics[model.name][n] = {}

            for i in range(num_task):
                data_sampler = get_data_sampler("iv", n_dims, normalize_Theta=normalize_Theta, mode=mode, q_truncated=q_truncated)
                task_sampler = get_task_sampler("iv_regression", n_dims, batch_size, normalize_w=normalize_w)
                U = torch.randn(batch_size, n_points, p)
                xzs, xs_p = gen_standard_iv(data_sampler, n_points, batch_size, U, scale_Theta)
                pred_mse = eval_batch(model, task_sampler, xzs, xs_p, U)
                coef_mse = eval_coef(model, task_sampler, xzs, U, delta=delta)

                all_pred_mse.append(pred_mse[:, -1].mean().item())
                all_coef_mse.append(coef_mse[:, -1].mean().item())

            metrics[model.name][n]['pred_mse'] = np.mean(all_pred_mse)
            metrics[model.name][n]['coef_mse'] = np.mean(all_coef_mse)

    return metrics

def eval_ivstrength(run_path, r_list, n=50, batch_size=64, n_dims=15, 
                                normalize_w=True, normalize_Theta=False, mode="linear",
                                q_truncated=None, num_task=500, delta=5, seed=1):
    # Evaluate MSE vs IV strength
    
    torch.random.manual_seed(seed)
    p = int(1/3 * n_dims)

    # Load the GPT model and its configuration
    gptmodel, conf = get_model_from_run(run_path, step=-1)
    gptmodel = gptmodel.cuda().eval()

    # Models to evaluate
    all_model = [IVRegressionModel(), LeastSquaresModel(), gptmodel]

    metrics = {}

    for model in all_model:
        metrics[model.name] = {}
        
        # Adjust batch size for GPT model
        if "gpt" in model.name:
            batch_size = int(batch_size / 4)
        
        # Loop over the list of r (IV strength)
        for r in tqdm(r_list):
            n_points = n + 1  # Include test sample

            all_pred_mse = []
            all_coef_mse = []
            metrics[model.name][r] = {}

            for i in range(num_task):
                # Data and task sampling
                data_sampler = get_data_sampler("iv", n_dims, normalize_Theta=normalize_Theta, q_truncated=q_truncated)
                task_sampler = get_task_sampler("iv_regression", n_dims, batch_size, normalize_w=normalize_w)
                
                # Generate random input data
                U = torch.randn(batch_size, n_points, p)
                xzs, xs_p = gen_standard_iv(data_sampler, n_points, batch_size, U, r)
                
                # Evaluate model predictions and coefficients
                pred_mse = eval_batch(model, task_sampler, xzs, xs_p, U)
                coef_mse = eval_coef(model, task_sampler, xzs, U, delta=delta)

                # Collect MSE values
                all_pred_mse.append(pred_mse[:, -1].mean().item())
                all_coef_mse.append(coef_mse[:, -1].mean().item())

            # Store the mean MSE values for this model and 'n'
            metrics[model.name][r]['pred_mse'] = np.mean(all_pred_mse)
            metrics[model.name][r]['coef_mse'] = np.mean(all_coef_mse)

    return metrics

def eval_endostrength(run_path, e_list, n=50, batch_size=64, n_dims=15, 
                                normalize_w=True, normalize_Theta=False, mode="linear",
                                q_truncated=None, num_task=500, delta=5, seed=1):
    # Evaluate MSE vs endogeneity strength
    
    torch.random.manual_seed(seed)
    p = int(1/3 * n_dims)

    # Load the GPT model and its configuration
    gptmodel, conf = get_model_from_run(run_path, step=-1)
    gptmodel = gptmodel.cuda().eval()

    # Models to evaluate
    all_model = [IVRegressionModel(), LeastSquaresModel(), gptmodel]

    metrics = {}

    for model in all_model:
        metrics[model.name] = {}
        
        # Adjust batch size for GPT model
        if "gpt" in model.name:
            batch_size = int(batch_size / 4)
        
        # Loop over the list of r (IV strength)
        for scale_u in tqdm(e_list):
            n_points = n + 1  # Include test sample

            all_pred_mse = []
            all_coef_mse = []
            metrics[model.name][scale_u] = {}

            for i in range(num_task):
                # Data and task sampling
                data_sampler = get_data_sampler("iv", n_dims, normalize_Theta=normalize_Theta, q_truncated=q_truncated)
                task_sampler = get_task_sampler("iv_regression", n_dims, batch_size, normalize_w=normalize_w)
                
                # Generate random input data
                U = torch.randn(batch_size, n_points, p) * scale_u
                xzs, xs_p = gen_standard_iv(data_sampler, n_points, batch_size, U)
                
                # Evaluate model predictions and coefficients
                pred_mse = eval_batch(model, task_sampler, xzs, xs_p, U)
                coef_mse = eval_coef(model, task_sampler, xzs, U, delta=delta)

                # Collect MSE values
                all_pred_mse.append(pred_mse[:, -1].mean().item())
                all_coef_mse.append(coef_mse[:, -1].mean().item())

            # Store the mean MSE values for this model and 'n'
            metrics[model.name][scale_u]['pred_mse'] = np.mean(all_pred_mse)
            metrics[model.name][scale_u]['coef_mse'] = np.mean(all_coef_mse)

    return metrics


def plot_mse_vs_sample_size(metrics, n_list, colors=None, ylim=None, ylim_inset=None, title=None, save_path=None):

    # Define a color map for the models if not provided
    if colors is None:
        colors = ['blue', 'green', 'red', 'orange', 'cyan']
    color_map = {}
    i = 0  # Index for color list

    plt.figure(figsize=(14, 10))

    # Loop through each model
    for model_name, points in metrics.items():
        if "iv" in model_name:
            if "Ridge" in model_name:
                model_name = "Ridge 2SLS"
            else:
                model_name = "2SLS"
        elif "OLS" in model_name:
            if "Ridge" in model_name:
                model_name = "Ridge OLS"
            else:
                model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"
        if model_name not in color_map:
            color_map[model_name] = colors[i % len(colors)]
            i += 1

        pred_mses = []
        coef_mses = []
        
        # Collect MSE values for each number of points
        for n in n_list:
            pred_mses.append(points[n]['pred_mse'])
            coef_mses.append(points[n]['coef_mse'])
        
        # Plotting the MSE values
        plt.plot(n_list, pred_mses, label=f'{model_name} ICPE', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        plt.plot(n_list, coef_mses, label=f'{model_name} MSE', linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        
    if ylim is None:
        ylim = (0, 0.7 * max([max(points[n]['pred_mse'], points[n]['coef_mse']) for points in metrics.values() for n in n_list]))
    if title is None:
        title = r'Error$^2$ vs In-context Sample Size'

    plt.title(title, fontsize=28, pad=20)
    plt.xlabel('Number of In-context Samples', fontsize=28)
    plt.ylabel(r'Error$^2$', fontsize=28)
    plt.ylim(ylim)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.legend(loc='upper right', fontsize=24)
    plt.grid(True)

    # Create an inset plot for zoom-in and position it outside the main plot in the lower-right corner
    ax_inset = inset_axes(plt.gca(), width="40%", height="40%", loc='lower right', 
                          bbox_to_anchor=(0.65, 0.1, 0.5, 0.5), bbox_transform=plt.gcf().transFigure)

    # Plotting the zoomed-in region on the inset plot
    for model_name, points in metrics.items():
        if "iv" in model_name:
            if "Ridge" in model_name:
                model_name = "Ridge 2SLS"
            else:
                model_name = "2SLS"
        elif "OLS" in model_name:
            if "Ridge" in model_name:
                model_name = "Ridge OLS"
            else:
                model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"

        pred_mses = []
        coef_mses = []

        for n in n_list:
            pred_mses.append(points[n]['pred_mse'])
            coef_mses.append(points[n]['coef_mse'])

        # Plot the zoomed-in region for MSEE
        ax_inset.plot(n_list, coef_mses, linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        ax_inset.plot(n_list, pred_mses, color=color_map[model_name], marker='', linewidth=3, markersize=10)

    # Set limits for the zoomed-in region
    if ylim_inset is None:
        ylim_inset = (0, 1)
    ax_inset.set_xlim(n_list[-1]-7, n_list[-1]+0.5)  # Adjust the x-limits to zoom in around these points
    ax_inset.set_ylim(ylim_inset)  # Adjust the y-limits to focus on lower MSEE values
    ax_inset.grid(True)

    # Optionally, customize inset appearance (ticks, labels, etc.)
    ax_inset.tick_params(axis='both', which='major', labelsize=12)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    else:
        plt.show()

def plot_mse_vs_ivstrength(metrics, r_list, colors=None, ylim=None, ylim_inset=None, title=None, save_path=None):

    # Define a color map for the models if not provided
    if colors is None:
        colors = ['blue', 'green', 'red', 'orange', 'cyan']
    color_map = {}
    i = 0  # Index for color list

    plt.figure(figsize=(14, 10))

    # Loop through each model
    for model_name, points in metrics.items():
        if "iv" in model_name:
            model_name = "2SLS"
        elif "OLS" in model_name:
            model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"
        if model_name not in color_map:
            color_map[model_name] = colors[i % len(colors)]
            i += 1

        pred_mses = []
        coef_mses = []
        
        # Collect MSE values for each number of points
        for r in r_list:
            pred_mses.append(points[r]['pred_mse'])
            coef_mses.append(points[r]['coef_mse'])
        
        # Plotting the MSE values
        plt.plot(r_list, pred_mses, label=f'{model_name} ICPE', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        plt.plot(r_list, coef_mses, label=f'{model_name} MSE', linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        
    if ylim is None:
        ylim = (0, 0.7 * max([max(points[r]['pred_mse'], points[r]['coef_mse']) for points in metrics.values() for r in r_list]))
    if title is None:
        title = r'Error$^2$ vs IV Strength'

    plt.title(title, fontsize=28, pad=20)
    plt.xlabel('Instrumental Strength', fontsize=28)
    plt.ylabel(r'Error$^2$', fontsize=28)
    plt.ylim(ylim)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.legend(loc='upper right', fontsize=24)
    plt.grid(True)

    # Create an inset plot for zoom-in and position it outside the main plot in the lower-right corner
    ax_inset = inset_axes(plt.gca(), width="40%", height="40%", loc='lower right', 
                          bbox_to_anchor=(0.65, 0.1, 0.5, 0.5), bbox_transform=plt.gcf().transFigure)

    # Plotting the zoomed-in region on the inset plot
    for model_name, points in metrics.items():
        if "iv" in model_name:
            model_name = "2SLS"
        elif "OLS" in model_name:
            model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"

        pred_mses = []
        coef_mses = []

        for r in r_list:
            pred_mses.append(points[r]['pred_mse'])
            coef_mses.append(points[r]['coef_mse'])

        # Plot the zoomed-in region for MSEE
        ax_inset.plot(r_list, coef_mses, linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        ax_inset.plot(r_list, pred_mses, color=color_map[model_name], marker='', linewidth=3, markersize=10)

    # Set limits for the zoomed-in region
    if ylim_inset is None:
        ylim_inset = (0, 1)
    ax_inset.set_xlim(r_list[-1]-0.35, r_list[-1]+0.03)  # Adjust the x-limits to zoom in around these points
    ax_inset.set_ylim(ylim_inset)  # Adjust the y-limits to focus on lower MSEE values
    ax_inset.grid(True)

    # Optionally, customize inset appearance (ticks, labels, etc.)
    ax_inset.tick_params(axis='both', which='major', labelsize=12)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    else:
        plt.show()

def plot_mse_vs_endostrength(metrics, e_list, colors=None, ylim=None, ylim_inset=None, title=None, save_path=None):

    # Define a color map for the models if not provided
    if colors is None:
        colors = ['blue', 'green', 'red', 'orange', 'cyan']
    color_map = {}
    i = 0  # Index for color list

    plt.figure(figsize=(14, 10))

    # Loop through each model
    for model_name, points in metrics.items():
        if "iv" in model_name:
            model_name = "2SLS"
        elif "OLS" in model_name:
            model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"
        if model_name not in color_map:
            color_map[model_name] = colors[i % len(colors)]
            i += 1

        pred_mses = []
        coef_mses = []
        
        # Collect MSE values for each number of points
        for scale_u in e_list:
            pred_mses.append(points[scale_u]['pred_mse'])
            coef_mses.append(points[scale_u]['coef_mse'])
        
        # Plotting the MSE values
        plt.plot(e_list, pred_mses, label=f'{model_name} ICPE', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        plt.plot(e_list, coef_mses, label=f'{model_name} MSE', linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        
    if ylim is None:
        ylim = (0, 0.7 * max([max(points[scale_u]['pred_mse'], points[scale_u]['coef_mse']) for points in metrics.values() for scale_u in e_list]))
    if title is None:
        title = r'Error$^2$ vs Endogeneity Level'

    plt.title(title, fontsize=28, pad=20)
    plt.xlabel('Endogeneity Level', fontsize=28)
    plt.ylabel(r'Error$^2$', fontsize=28)
    plt.ylim(ylim)
    plt.tick_params(axis='both', which='major', labelsize=22)
    plt.legend(loc='upper left', fontsize=24)
    plt.grid(True)

    # Create an inset plot for zoom-in and position it outside the main plot in the lower-right corner
    ax_inset = inset_axes(plt.gca(), width="40%", height="40%", loc='lower right', 
                          bbox_to_anchor=(0.65, 0.1, 0.5, 0.5), bbox_transform=plt.gcf().transFigure)

    # Plotting the zoomed-in region on the inset plot
    for model_name, points in metrics.items():
        if "iv" in model_name:
            model_name = "2SLS"
        elif "OLS" in model_name:
            model_name = "OLS"
        elif "gpt" in model_name:
            model_name = "Transformer"

        pred_mses = []
        coef_mses = []

        for scale_u in e_list:
            pred_mses.append(points[scale_u]['pred_mse'])
            coef_mses.append(points[scale_u]['coef_mse'])

        # Plot the zoomed-in region for MSEE
        ax_inset.plot(e_list, coef_mses, linestyle='--', color=color_map[model_name], marker='', linewidth=3, markersize=10)
        ax_inset.plot(e_list, pred_mses, color=color_map[model_name], marker='', linewidth=3, markersize=10)

    # Set limits for the zoomed-in region
    if ylim_inset is None:
        ylim_inset = (0, 1)
    ax_inset.set_xlim(e_list[-1]-0.35, e_list[-1]+0.03)  # Adjust the x-limits to zoom in around these points
    ax_inset.set_ylim(ylim_inset)  # Adjust the y-limits to focus on lower MSEE values
    ax_inset.grid(True)

    # Optionally, customize inset appearance (ticks, labels, etc.)
    ax_inset.tick_params(axis='both', which='major', labelsize=12)

    # Save or display the plot
    if save_path:
        plt.savefig(save_path, bbox_inches='tight')
    else:
        plt.show()




### Set Model Path

In [ ]:
run_dir = "../TrainedTF"
run_name = "iv_regression"
run_id = "Loop=10_N=51_d=15_nhead=12_D=80_L=2"

run_path = os.path.join(run_dir, run_name, run_id)
n_list = range(20, 51, 2)

### Evaluation (May take a few hours) 

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, mode = "quadratic", num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n_quadratic.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, mode = "non-linear", num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n_non-linear.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, q_truncated = 3, num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n_underdetermined.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, mode = "multicollinearity", num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n_multicollinearity.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
metrics = evaluate_models(run_path = run_path, n_list = n_list, mode = "multicollinearity-heavy", num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_n_multicollinearity-heavy.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
r_list = np.arange(0.1, 2.1, 0.1)
metrics = eval_ivstrength(run_path = run_path, r_list = r_list, num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_ivstrength.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

In [ ]:
e_list = np.arange(0.1, 2.1, 0.1)
metrics = eval_endostrength(run_path = run_path, e_list = e_list, num_task=500)
store_path = os.path.join(run_path, "results/mse_vs_endostrength.pkl")
with open(store_path, "wb") as file:
    pickle.dump(metrics, file)

### Plotting

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,6), save_path=save_path)

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n_quadratic.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n_quadratic.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,12), ylim_inset=(0,0.5), title=r"Error$^2$ vs In-Context Sample Size (Quadratic IV)", save_path=save_path)

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n_non-linear.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n_non-linear.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,12), ylim_inset=(0,0.6), title=r"Error$^2$ vs In-Context Sample Size (Non-Linear IV)", save_path=save_path)

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n_underdetermined.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n_underdetermined.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,5), ylim_inset=(0,1), title=r"Error$^2$ vs In-Context Sample Size (q<p)", save_path=save_path)

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n_multicollinearity.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n_multicollinearity.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,12.8), ylim_inset=(0,1), title=r"Error$^2$ vs In-Context Sample Size (Multicollinearity)", save_path=save_path)

In [ ]:
load_path = os.path.join(run_path, "results/mse_vs_n_multicollinearity-heavy.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_n_multicollinearity-heavy.png")
plot_mse_vs_sample_size(metrics, n_list, ylim=(0,6), ylim_inset=(0,1), title=r"Error$^2$ vs In-Context Sample Size (Heavy Multicollinearity)", save_path=save_path)

In [ ]:
r_list = np.arange(0.1, 2.1, 0.1)
load_path = os.path.join(run_path, "results/mse_vs_ivstrength.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_ivstrength.png")
plot_mse_vs_ivstrength(metrics, r_list, ylim=(0,4.8), save_path=save_path)

In [ ]:
r_list = np.arange(0.1, 2.1, 0.1)
load_path = os.path.join(run_path, "results/mse_vs_endostrength.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)
save_path = os.path.join(run_path, "figures/mse_vs_endostrength.png")
plot_mse_vs_endostrength(metrics, r_list, ylim=(0,6), save_path=save_path)

### Real World Dataset Estimation

In [ ]:
from sas7bdat import SAS7BDAT

def calculate_beta_gpt(df, gptmodel, delta=1):
    # Variables of interest
    y = torch.tensor(df['WEEKSM'].to_numpy(), dtype=torch.float32)  # Mother's hours of work per week
    z = torch.tensor(df['SEX2ND'].to_numpy(), dtype=torch.float32)  # Indicator for whether the first and second child have the same sex
    x = torch.tensor(df['KIDCOUNT'].to_numpy(), dtype=torch.float32)  # Number of children
    
    xz = torch.zeros((len(df), 15), dtype=torch.float32) 
    xz[:, 0] = x
    xz[:, 5] = z

    xz = xz.unsqueeze(0)
    z = z.unsqueeze(0)
    x = x.unsqueeze(0)
    y = y.unsqueeze(0)

    xz1 = xz.clone()
    y_hat1 = gptmodel(xz1.to("cuda"), y.to("cuda"), inds=[y.shape[1] - 1]).detach()
    
    # Compute perturbed prediction
    xz2 = xz.clone()
    xz2[:, -1, 0] += delta
    y_hat2 = gptmodel(xz2.to("cuda"), y.to("cuda"), inds=[y.shape[1] - 1]).detach()
    
    # Calculate beta_gpt
    beta_gpt = (y_hat2.to("cpu") - y_hat1.to("cpu")) / delta
    return beta_gpt

def estimate_beta(df, gptmodel, n=50, delta=5, repetitions=100, seed=1):
    beta_gpt_list = []
    n = n + 1 # Include test sample
    
    for _ in range(repetitions):
        sampled_df = df.sample(n=51, random_state=seed, replace=False)
        # sampled_df = df
        
        df0 = sampled_df[:n]
        beta = calculate_beta_gpt(df0, gptmodel, delta=delta)
        beta_gpt_list.append(beta)
        seed += 1
        
    # Calculate the average beta_gpt across all repetitions
    beta_gpt = np.median(beta_gpt_list, axis=0)
    beta_gpt_list = np.array([beta_gpt_list[i] for i in range(len(beta_gpt_list))])[:,0,0]
    return beta_gpt, beta_gpt_list

# Load model
gptmodel, conf = get_model_from_run(run_path, step=-1)
gptmodel = gptmodel.cuda().eval()
delta = 5

# Set the data path
file_path = os.path.join(run_path, "AngEv98/m_d_806.sas7bdat")

# Read only the first 10000 rows
with SAS7BDAT(file_path) as file:
    chunk = []
    for i, row in enumerate(file):
        if i == 0:
            # Use the first row as column names
            columns = row
        else:
            chunk.append(row)
        if len(chunk) == 20000: # Read only the first 20000 rows
            break

# Preprocessing
data = pd.DataFrame(chunk, columns=columns)
data = data[(data['WEEKSM'] != '00') & (data['SEX2ND'] != '') & (data['STATE']=='01')]
data['SEX2ND'] = data['SEX2ND'].astype(float).to_numpy() 
data['KIDCOUNT'] = data['KIDCOUNT'].astype(float).to_numpy() 
data['WEEKSM'] = data['WEEKSM'].astype(float).to_numpy() / 52

z = torch.tensor(data["SEX2ND"].to_numpy(), dtype=torch.float32)  # Indicator for whether the first and second child have the same sex
x = torch.tensor(data['KIDCOUNT'].to_numpy(), dtype=torch.float32)  # Number of children
y = torch.tensor(data['WEEKSM'].to_numpy(), dtype=torch.float32)  # Mother's number of working weeks per year

z_with_intercept = torch.cat([torch.ones_like(z).view(-1, 1), z.view(-1, 1)], dim=1)
Theta = torch.linalg.lstsq(z_with_intercept, x.view(-1, 1)).solution
x_hat = z_with_intercept @ Theta
x_hat_with_intercept = torch.cat([torch.ones_like(x_hat).view(-1, 1), x_hat.view(-1, 1)], dim=1)
beta_2sls = torch.linalg.lstsq(x_hat_with_intercept, y.view(-1, 1)).solution[1]
beta_ols = torch.linalg.lstsq(torch.cat([torch.ones_like(x.view(-1, 1)), x.view(-1, 1)], dim=1), y.view(-1, 1)).solution[1]

beta_gpt, beta_gpt_list = estimate_beta(data, gptmodel, n=50, delta=5, repetitions=500)


store_path = os.path.join(run_path, "results/labor_supply_estimate.pkl")
with open(store_path, "wb") as file:
    pickle.dump(beta_gpt_list, file)


In [ ]:
load_path = os.path.join(run_path, "results/labor_supply_estimate.pkl")
with open(load_path, "rb") as file:
    metrics = pickle.load(file)

save_path = os.path.join(run_path, "figures/labor_supply_estimate_boxplot.png")

sns.boxplot(data=metrics, color='grey')

# Enhancing labels and title
# plt.xlabel('Number of In-Context Samples (n=50)', fontsize=14, labelpad=10)
plt.ylabel(r'Estimated $\beta$', fontsize=14, labelpad=10)
plt.title('Boxplot of Estimated Coefficient', fontsize=16, pad=15)
plt.axhline(y=beta_ols, color='green', linestyle='--', linewidth=1.5, label=fr'$\beta_{{OLS}} = {float(beta_ols):.3f}$')
plt.axhline(y=beta_2sls, color='blue', linestyle='--', linewidth=1.5, label=fr'$\beta_{{2SLS}} = {float(beta_2sls):.3f}$')
plt.axhline(y=np.median(metrics), color='red', linestyle='--', linewidth=1.5, label=fr'$\beta_{{GPT}} = {np.median(metrics):.3f}$')

# Display the plot
plt.ylim(-0.17,0.03)
plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig(save_path)
plt.show()
